In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import seaborn as sns

# Mount Google Drive (specific to Google Colab)
# from google.colab import drive
# drive.mount('/content/gdrive')
# data = pd.read_csv('/content/gdrive/MyDrive/RoadSense/01_tracks-2.csv')

tracks1 = pd.read_csv('01_tracks.csv')
tracks2 = pd.read_csv('02_tracks.csv')
tracks3 = pd.read_csv('03_tracks.csv')
tracks4 = pd.read_csv('04_tracks.csv')
tracks5 = pd.read_csv('05_tracks.csv')

def get_lane_changed_ID(data):
    unique_ID = len(data['id'].unique())
    changedID = list()
    for i in range(1, unique_ID):
        vehicle = data.loc[data["id"] == i]
        if vehicle['laneId'].iloc[0] < 3.5 and vehicle['laneId'].iloc[0] - vehicle['laneId'].iloc[-1] > 0:
            changedID.append(i)
        if vehicle['laneId'].iloc[0] < 3.5 and vehicle['laneId'].iloc[0] - vehicle['laneId'].iloc[-1] < 0:
            changedID.append(i)
        if vehicle['laneId'].iloc[0] > 3.5 and vehicle['laneId'].iloc[0] - vehicle['laneId'].iloc[-1] > 0:
            changedID.append(i)
        if vehicle['laneId'].iloc[0] > 3.5 and vehicle['laneId'].iloc[0] - vehicle['laneId'].iloc[-1] < 0:
            changedID.append(i)
    return changedID

data = combined_data = pd.concat([tracks1, tracks2, tracks3, tracks4, tracks5], ignore_index=False)

LaneChangedID = get_lane_changed_ID(data)

data['lane_change'] = 0
data.loc[data['id'].isin(LaneChangedID), 'lane_change'] = 1

columns_to_drop = [
    'frontSightDistance', 'backSightDistance', 'dhw', 'thw', 'ttc',
    'precedingXVelocity', 'precedingId', 'followingId', 'leftPrecedingId',
    'leftAlongsideId', 'leftFollowingId', 'rightPrecedingId', 'rightAlongsideId',
    'rightFollowingId', 'laneId', 'width', 'height', 'frame'
]
data = data.drop(columns_to_drop, axis=1)

grouped = data.groupby('id')
sequences = [group.drop('id', axis=1).values for _, group in grouped]

max_sequence_length = max(len(s) for s in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', dtype='float32')

# Convert labels to one-hot encoded format
labels = np.array([group['lane_change'].iloc[0] for _, group in grouped])
one_hot_labels = to_categorical(labels, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2, random_state=42)

model = tf.keras.Sequential([
    Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(max_sequence_length, padded_sequences.shape[2]), dropout=0.2)),
    Bidirectional(tf.keras.layers.LSTM(30, dropout=0.2)),
    tf.keras.layers.Dense(3, activation='softmax')
])

def F1Score(y_true, y_pred):
    # Calculating Precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    # Calculating Recall
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    # Calculating F1 Score
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall(), F1Score])

class_weights = {0: 1, 1: 2, 2: 2}

history = model.fit(X_train, y_train, epochs=10, batch_size=12, validation_split=0.2, class_weight=class_weights)

loss, accuracy, precision, recall, f1_score = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")
print(f"Test Precision: {precision*100:.2f}%")
print(f"Test Recall: {recall*100:.2f}%")
print(f"Test F1 Score: {f1_score:.4f}")

f1_score = F1Score(y_test, model.predict(X_test))
print(f"Test F1 Score: {f1_score:.4f}")
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

Epoch 1/10


2023-12-14 16:58:03.021194: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-14 16:58:03.021224: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-14 16:58:03.021234: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-14 16:58:03.021263: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-14 16:58:03.021277: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-12-14 16:58:06.629250: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


65/65 [==============================] - 61s 826ms/step - loss: 0.7466 - accuracy: 0.8134 - precision: 0.8242 - recall: 0.7722 - F1Score: 0.7783 - val_loss: 0.4711 - val_accuracy: 0.8308 - val_precision: 0.8308 - val_recall: 0.8308 - val_F1Score: 0.8088
Epoch 2/10
65/65 [==============================] - 52s 796ms/step - loss: 0.5580 - accuracy: 0.8250 - precision: 0.8250 - recall: 0.8250 - F1Score: 0.8239 - val_loss: 0.4557 - val_accuracy: 0.8308 - val_precision: 0.8308 - val_recall: 0.8308 - val_F1Score: 0.8088
Epoch 3/10
65/65 [==============================] - 52s 802ms/step - loss: 0.5028 - accuracy: 0.8250 - precision: 0.8250 - recall: 0.8250 - F1Score: 0.8244 - val_loss: 0.4042 - val_accuracy: 0.8308 - val_precision: 0.8308 - val_recall: 0.8308 - val_F1Score: 0.8088
Epoch 4/10
 2/65 [..............................] - ETA: 46s - loss: 0.3560 - accuracy: 0.8750 - precision: 0.8750 - recall: 0.8750 - F1Score: 0.8750

In [8]:
np.shape(l_to_r)

(731,)